# Analise e modelo para predição de vida útil restante de motores

---

Variaveis:

 - **dataset**: string

 Nome do arquivo h5 usado para a analise.

 - **carregar_modelos_github**: boolean

   - Baixa o dataset e os modelos pré-treinados no repo do github. Opção mais rápida, porém não suporta escrita de novos modelos.

 - **usar_gdrive**: boolean
 
   - Buscará o arquivo `dataset` e os modelos no Google Drive, dentro do diretório `Meu disco > tcc-machine-learning`. É necessário permissão de acesso ao GDrive e também que os arquivos sejam disponibilizados previamente no diretório. Está opção suporta também a escrita de novos modelos no caminho citado.

 - Caso ambos `carregar_modelos_github` e `usar_gdrive` sejam `False` o *notebook* baixará o ZIP completo (16gb) do repositório da NASA e extrairá o arquivo `dataset` toda vez que for iniciado. Este processo demora entre 20 e 50 minutos, dependendo da velocidade da rede entre Google e o repositório. Além disso, esta opção não oferece nenhum modelo pré-treinado.

In [ ]:
dataset = 'N-CMAPSS_DS02-006.h5'
carregar_modelos_github = True
usar_gdrive = False

## 01 - Baixar os datasets e extrai-los

Link disponível em:

 - https://www.nasa.gov/content/prognostics-center-of-excellence-data-set-repository
 - https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/

Obs.: é recomendado ao menos 60gb de espaço livre para poder baixar e extrair os datasets.

In [ ]:
from google.colab import drive

def monta_drive():
  raiz = '/content/drive'
  drive.mount(raiz)
  caminho_tcc = f'{raiz}/MyDrive/tcc-machine-learning'
  return caminho_tcc


In [ ]:
if usar_gdrive:
  caminho_tcc = monta_drive()
  caminho_dataset = f'/{dataset}'
  !rsync -h --progress $caminho_tcc/$dataset $caminho_dataset
elif carregar_modelos_github:
  caminho_dataset = f'/{dataset}'
  !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/N-CMAPSS_DS02-006.h5.gz | gzip -d -c > $caminho_dataset
else:
  caminho_dataset = f'data_set/{dataset}'
  !curl https://phm-datasets.s3.amazonaws.com/NASA/17.+Turbofan+Engine+Degradation+Simulation+Data+Set+2.zip | jar xv
  !unzip '17. Turbofan Engine Degradation Simulation Data Set 2/data_set.zip'
  !rm -fr '17. Turbofan Engine Degradation Simulation Data Set 2/'


## 02 - Importar bibliotecas e carregar os dados

In [ ]:
!pip install joblib

from collections import namedtuple
from datetime import datetime

import sys

# bibliotecas do serialização
import h5py
from joblib import dump, load

# bibliotecas numéricas
import numpy as np
import pandas as pd
from scipy.stats import randint, loguniform
from pandas import DataFrame

# ferramentas de visualização
import matplotlib
import matplotlib.pyplot as plt

# ferramentas de machine learning
import sklearn
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.base import RegressorMixin
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

# exibir os gráficos no jupyter
%matplotlib inline

# formatar os pontos flutuantes para exibir 5 casas decimais
pd.set_option('display.float_format', '{:.5f}'.format)

# exibe diagramas interativos dos estimadores
sklearn.set_config(display="diagram")

print("Python", sys.version)
print("Numpy", np.__version__)
print("Pandas", pd.__version__)
print("Matplotlib", matplotlib.__version__)
print("Sklearn", sklearn.__version__)

In [ ]:
def renomeia_descr(df):
   return df.rename(columns={
        "alt": "Altitude",
        "Mach": "Flight Mach number",
        "TRA": "Throttle–resolver angle",
        "T2": "Total temperature at fan inlet",
   })
   
def renomeia_sensores(df):
   return df.rename(columns={
        "Wf": "Fuel flow",
        "Nf": "Physical fan speed",
        "Nc": "Physical core speed",
        "T24": "Total temperature at LPC outlet",
        "T30": "Total temperature at HPC outlet",
        "T48": "Total temperature at HPT outlet",
        "T50": "Total temperature at LPT outlet",
        "P15": "Total pressure in bypass-duct",
        "P2": "Total pressure at fan inlet",
        "P21": "Total pressure at fan outlet",
        "P24": "Total pressure at LPC outlet",
        "Ps30": "Static pressure at HPC outlet",
        "P40": "Total pressure at burner outlet",
        "P50": "Total pressure at LPT outlet",
    })

with h5py.File(caminho_dataset, 'r') as hdf:
  def pega_df(dataset, variavel, nome_colunas = None):
    if not nome_colunas:
        nome_colunas = np.array(hdf.get(f'{variavel}_var'), dtype=np.unicode_)
    dataframe_numpy = np.array(hdf.get(f'{variavel}_{dataset}'))
    return pd.DataFrame(dataframe_numpy, columns=nome_colunas)
  
  A_dev   = pega_df('dev', 'A').astype('int')
  W_dev   = renomeia_descr(pega_df('dev', 'W'))
  X_s_dev = renomeia_sensores(pega_df('dev', 'X_s'))
  Y_dev   = pega_df('dev', 'Y', ['RUL'])
  dev     = pd.concat([A_dev, W_dev, X_s_dev, Y_dev], ignore_index=False, axis=1)

  A_test   = pega_df('test', 'A').astype('int')
  W_test   = renomeia_descr(pega_df('test', 'W'))
  X_s_test = renomeia_sensores(pega_df('test', 'X_s'))
  Y_test   = pega_df('test', 'Y', ['RUL'])
  test     = pd.concat([A_test, W_test, X_s_test, Y_test], ignore_index=False, axis=1)

## 03 - Analise exploratória

### 03.1 - Descrição dos dados

In [ ]:
A_dev.describe().T

In [ ]:
W_dev.describe().T

In [ ]:
X_s_dev.describe().T

In [ ]:
Y_dev.describe().T

### 03.2 - Informações

#### 03.2.1 - Total de eventos por unidade

In [ ]:
dev.groupby('unit').size()

#### 03.2.2 - Quantidade de ciclos e RUL por unidade

O número de ciclos é uma unidade acima do RUL, visto que o último ciclo tem RUL = 0

In [ ]:
dev.groupby('unit')[['cycle', 'RUL']].max()

#### 03.2.3 - Quantidade de amostras por ciclo

In [ ]:
quantidade_eventos_ciclos = dev.groupby(['unit','cycle']).cycle.size()
quantidade_eventos_ciclos.groupby('unit').describe()

#### 03.2.4 - Verificar se algum item é nulo (NaN - not a number)

0 rows = nenhum item nulo

In [ ]:
dev[dev.isna().any(axis=1)]

#### 03.2.5 - Quais colunas permanecem constantes por unidades?

In [ ]:
itens_sem_varianca_unit = (dev.groupby(['unit'], as_index = False).std() == 0.0).all(axis = 0)
itens_sem_varianca_unit = set(itens_sem_varianca_unit[itens_sem_varianca_unit].keys())
pd.DataFrame(itens_sem_varianca_unit)

#### 03.2.6 - Quais colunas permanecem constantes dentro dos ciclos?

In [ ]:
itens_sem_varianca_ciclos = (dev.groupby(['unit', 'cycle'], as_index = False).std() == 0.0).all(axis = 0)
itens_sem_varianca_ciclos = set(itens_sem_varianca_ciclos[itens_sem_varianca_ciclos].keys())
pd.DataFrame(itens_sem_varianca_ciclos)

#### 03.2.7 - Quais colunas alteram somente entre ciclos?

In [ ]:
pd.DataFrame(itens_sem_varianca_ciclos.difference(itens_sem_varianca_unit))

#### 03.2.8 - Em qual ciclo o status de saúde do motor (Hs) muda de saudável (1) para não saudável (0)?

In [ ]:
pd.DataFrame(A_dev[ A_dev["hs"] == 0 ].groupby("unit")["cycle"].min())

### 03.3 Gráficos

#### 03.3.1 - Funções auxiliares

In [ ]:
## gerar contadores:
## 1 - com porcentagem de conclusão do ciclo (indice_linha_pct)
## 2 - porcentagem acumulativa e fracionada de ciclo (ciclo_acc)
## 3 - RUL fracionado (rul_frac)

## agrupa os dados por unidade e ciclo. Desta forma diferentes ciclos são tratados separadamente.
dados_por_ciclo = dev.groupby([ 'unit', 'cycle'])

## cria coluna com contador 1, 2,...,n dentro de cada ciclo/unidade
dev['indice_linha'] = dados_por_ciclo.cumcount()

## pega o total de linhas em cada ciclo/unidade 
dev['total_linhas'] = dados_por_ciclo['indice_linha'].transform(np.count_nonzero)

## transforma em porcentagem de conclusão do ciclo. 
dev['indice_linha_pct'] = dev['indice_linha']  / dev['total_linhas']

## adiciona ao acumulador
## Para tornar mais visível, transformamos 100% de conclusão do voo em 99%
## desta forma, ao final do ciclo 5 temos indice_linha_pct = 5.99 e não 6.00
dev['ciclo_acc'] = dev['cycle'] + dev['indice_linha_pct'] * 0.99
dev['RUL_frac'] = dev['RUL'] + (0.99 - dev['indice_linha_pct'] * 0.99)

# dev = dev.drop(columns=['indice_linha', 'total_linhas', 'indice_linha_pct'])

In [ ]:
# gera um gráfico plotando todos os ciclos de uma unidade

def gera_grafico(ax, num_unidade, coluna):
    unidade = dev[dev['unit'] == num_unidade]
    ciclo_max = unidade['cycle'].max()

    for i in range(1, ciclo_max + 1):
        ciclo = unidade[unidade['cycle'] == i ]
        x = ciclo['indice_linha']
        y = ciclo[coluna]
        alpha = 0.05 + (i / ciclo_max) * 0.95
        ax.plot(x, y, label=i, alpha=alpha)
    ax.set_title(coluna)

#### 03.3.2 - A (descrição)

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 7))
fig.subplots_adjust(hspace=0.4)

gera_grafico(ax1, 2, 'Altitude')
gera_grafico(ax2, 2, 'Flight Mach number')
	
gera_grafico(ax3, 2, 'Throttle–resolver angle')
gera_grafico(ax4, 2, 'Total temperature at fan inlet')

#### 03.3.3 - Correlação dentro de um ciclo

In [ ]:
num_unidade = 2
ciclo = 1

## removemos as colunas sem alteração ou que não fazem parte da predição de correlação
unidade = dev[ (dev['unit'] == num_unidade) & (dev['cycle'] == ciclo)].drop(columns=['unit', 'cycle', 'Fc', 'hs', 'RUL', 'indice_linha', 'total_linhas', 'indice_linha_pct', 'ciclo_acc', 'RUL_frac'])

correlacao = unidade.corr()
correlacao.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1)

In [ ]:
# Mostrar a correlação somente com os dados de W_dev
remover_colunas = set(unidade.columns).difference(set(W_dev.columns))
correlacao.drop(columns=remover_colunas).style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1)

Nota: é possível ver que `Total pressure in bypass-duct` tem correlação de 1 com `Total pressure at fan outlet`

## 04 - Modelagem

### 04.01 - Diagrama do modelo

Os items em **vermelho** são usados somente durante o treino.

[![](https://mermaid.ink/img/pako:eNqlVU1u00AUvsrIKEoqJbSJAVEjVari7urEsoWEZLMYPM_JqM6MNR4TSpotB0C9QGEB98i-h-AkPNtJajtBCGFLyfP7vvc_PysjkgwMy-h0VoQLri2y6saJXEZzqnQXv0g3ytVHQLG7kEJqKeBdl6yLt9MJRSj2bHLthYLgk-UfZoqmc3IltKKMBtv_9xVcPLYf9BhlMgtDkYHIpILspIa706CXUkUXoFVJkikoGnEpKG8QvbfXQQ9_6rpM3yZQICgpeQOWAtavxMGSMz23RumnfiQTqeoQoxmWoeitRUxiVu5AsEpwp4PBxcSdNiIPng8u7rQCLuSdsy3e9uvaie23euLyFBIuINgJ9cT3jRsGV5nePMy4JMMaoXjQ5Wo1kWpBE_6Zbn5svksyydbrFsudHrKWbZY9QV82iAavTXL8Hk1mUnG9kMQ_OcimqNfxD8IPDrWOj0qM2dRW43J8EvMksZZzruEYASP9_zwbM222fPTU8lGr5ZfuOAgNOlN5iitSaIkLMpWKRDxKCtntPd5HJ-TXl6_k8WdotKzHNhqPNw9JlCeSMEADxjHUNxFxih92z--T8db8vmFepnosWfMpWbMVzvGCp3F5B7ngBLCcg_qKaXlN7dEpeUen1GipXUx5bNcUk7amXB37NPbW-_p8yhmtbbXx5XVwihsOuxXRoo94dJzWSnM89Few2g7LpK-GVdLP4FXMABrQaAedxy_j8wZkbqE4BjNqOqzOsy3OzPgsjuv4bmvvHLwu3jqhrG-HnsFZ_KJC0ejGLxnDP651M_30ps0e_RPb_Avb6BsLwBOBM7wYVoV1aOg5LCA0LBQZxDRPdGiEYo1Ummvp34rIsLTKoW_kKaMabI7bhS4MK6ZJhlpgXEvlVJdNeeesfwMbQ_5Z?type=jpg)](https://mermaid.live/edit#pako:eNqlVU1u00AUvsrIKEoqJbSJAVEjVari7urEsoWEZLMYPM_JqM6MNR4TSpotB0C9QGEB98i-h-AkPNtJajtBCGFLyfP7vvc_PysjkgwMy-h0VoQLri2y6saJXEZzqnQXv0g3ytVHQLG7kEJqKeBdl6yLt9MJRSj2bHLthYLgk-UfZoqmc3IltKKMBtv_9xVcPLYf9BhlMgtDkYHIpILspIa706CXUkUXoFVJkikoGnEpKG8QvbfXQQ9_6rpM3yZQICgpeQOWAtavxMGSMz23RumnfiQTqeoQoxmWoeitRUxiVu5AsEpwp4PBxcSdNiIPng8u7rQCLuSdsy3e9uvaie23euLyFBIuINgJ9cT3jRsGV5nePMy4JMMaoXjQ5Wo1kWpBE_6Zbn5svksyydbrFsudHrKWbZY9QV82iAavTXL8Hk1mUnG9kMQ_OcimqNfxD8IPDrWOj0qM2dRW43J8EvMksZZzruEYASP9_zwbM222fPTU8lGr5ZfuOAgNOlN5iitSaIkLMpWKRDxKCtntPd5HJ-TXl6_k8WdotKzHNhqPNw9JlCeSMEADxjHUNxFxih92z--T8db8vmFepnosWfMpWbMVzvGCp3F5B7ngBLCcg_qKaXlN7dEpeUen1GipXUx5bNcUk7amXB37NPbW-_p8yhmtbbXx5XVwihsOuxXRoo94dJzWSnM89Few2g7LpK-GVdLP4FXMABrQaAedxy_j8wZkbqE4BjNqOqzOsy3OzPgsjuv4bmvvHLwu3jqhrG-HnsFZ_KJC0ejGLxnDP651M_30ps0e_RPb_Avb6BsLwBOBM7wYVoV1aOg5LCA0LBQZxDRPdGiEYo1Ummvp34rIsLTKoW_kKaMabI7bhS4MK6ZJhlpgXEvlVJdNeeesfwMbQ_5Z)


### 04.02 Fluxo do algoritmo

O algoritmo é dividido em 3 estágios e compara os valores de $X_s$ com valores estimados para sensores de componentes novos (primeiros $N$ ciclos, com pouco desgaste). Como os valores de $X_s$ tendem a se distanciar dos valores simulados a medida em que há aumento do desgaste, é possível usar essa diferença para calcular a vida útil restante.

 - **Estágio 1**
 
  1. É treinado um algoritmo $S$ que usa os parametros operacionais $W$ para estimar valores de sensores dos componentes com degradação natural para essas configurações de voo. Ou seja, dado um vetor $w\ \in\ W$, o resultado de $S(w)$ é um vetor $x_c$ com valores aproximados ao respectivo $x_s\ \in\ X_s$ sem degradação. Por conta do ruído nos dados originais, o valor poderá apresentar alguma variação $R_1$.<br/>

  <!-- $$ S(W) = X_c\ \approx\ X_s + R_1 $$ -->
  $$ W\ \overset{S}{\longrightarrow}\ X_c\ \approx\ X_s + R_1 $$ <br/>

  2. O algoritmo usado em $S$ tem como entrada e saída vetores numéricos $[v_{n}^{1}, ..., v_{n}^{m}] \subset\ [0, 1]^{m}$, denominado vetor normalizado. Por conta disso, durante o treino, a função de normalização $N_w$ é calibrada para transformar os seus valores $W$ de entrada em um vetor compatível $W_{normal}\ $ e a função $N_s$ é calibrada para transformar os valores de $X_s$ em $S_{normal\ }$.<br/><br/>
  <!-- $$ W_{normal} \in\ {[0, 1]}^{m_w}\ =\ N_w(W \subset\ \mathbb{R}^{m_w}) $$ -->
  <!-- $$ S_{normal} \in\ {[0, 1]}^{m_s} \ =\ N_s(X_s \subset\ \mathbb{R}^{m_s}) $$ -->
  $$ W\ \subset\ \mathbb{R}^{m_w}\ \overset{N_w}{\longrightarrow}\ W_{normal}\ \subset\ {[0, 1]}^{m_w} $$ <br/>
  $$ X_s \subset\ \mathbb{R}^{m_s}\ \overset{N_s}{\longrightarrow}\ S_{normal}\ \subset\ {[0, 1]}^{m_s} $$ <br/>
  $$ W\ \overset{N_w}{\longrightarrow} W_{normal}\ \ \overset{S}{\longrightarrow}\ S_{normal} + R_1 $$ <br/>
  3. Visto que $ S_{normal}\ \approx\ S(W_{normal\ }) $, durante a fase de inferência a função inversa $N_s^{-1}$ é usada para tranformar a saída do algoritmo $S$ em valores na mesma faixa de $X_s$.<br/><br/>

  $$ W\ \overset{N_w}{\longrightarrow} W_{normal}\ \ \overset{S}{\longrightarrow}\ S_{normal\ }\ \ \overset{N_s^{-1}}{\longrightarrow}\ X_c$$ <br/>

 - **Estágio 2**

  1. É usada uma métrica $D$ para o cálculo de distância entre cada valor $x_s$ e o respectivo $x_c$ calculado. Quanto maior a degradação do componente, maior será a distância entre os valores.

 $$ X_s,\ X_c\ \overset{D}{\longrightarrow}\ \Delta $$<br/>

  2. O resultado $\Delta$ é particionado em subconjuntos contendo somente dados de um mesmo ciclo para cada unidade. Cada subconjunto é então reduzido para estimar o progresso do desgaste geral através da função $P$.<br/><br/>

 $$ \Delta_{\ \subset\ ciclo}\ \overset{P}{\longrightarrow}\ \Psi_{ciclo} $$<br/>

 - **Estágio 3**

  1. Um segundo algoritmo $R$ é treinado para mapear os valores de $\Psi_{ciclo}$ para a vida útil restante (RUL. Por conta do ruído dos sensores, os valores apresentarão variação $R_2$ em relação ao RUL real.

 $$ \Psi_{ciclo}\ \overset{R}{\longrightarrow}\ RUL_{calculado} + R_2 $$<br/>

### 04.03 Funções auxiliares

In [ ]:
def verifica_scores(original: np.ndarray, predicao: np.ndarray):
    r2 = r2_score(original, predicao)
    rmse = np.sqrt(mean_squared_error(original, predicao))
    return rmse, r2


def print_scores(info, original, predicao):
    rmse, r2 = verifica_scores(original, predicao)
    r2 = r2
    print(f'{info} - RMSE: {rmse:.3f}, R2: {r2:.2f}')

def print_data():
    print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))

def pega_reducao_percent(a, b):
  100 * (a - b) / a

def salvar_modelo(nome, modelo):
    data = datetime.today().strftime('%Y%m%d_%H%M%S')
    caminho_com_data = f'/{data}_{nome}.joblib'

    print(f'Salvando modelo em {caminho_com_data}')
    dump(modelo, caminho_com_data)

    print('Copiando para Google Drive')
    !rsync -h --progress $caminho_com_data $caminho_tcc/

def plot_motores(original, predicao1, predicao2 = None, legendas = ["14 sensores", "5 sensores"]):
    unidades = unidade_test.unique()
    ax, p = plt.subplots(1, len(unidades), figsize=(15, 4))
    for i, unit in enumerate(unidades):
        k = unidade_test == unit
        p[i].plot(original[k].reset_index().drop(columns=['index']), label="real", alpha=1)
        p[i].plot(predicao1[k], label=legendas[0], alpha=0.75)
        if predicao2 is not None:
          p[i].plot(predicao2[k], label=legendas[1], alpha=0.75)
        p[i].set_title(f'Motor {unit}')
        p[i].legend(loc='upper right', fancybox=True, shadow=True, ncol=1)
        p[i].set(xlabel='ciclos de voo', ylabel='ciclos restantes (RUL)')


### 04.04 - Código auxiliar para treino do modelo S e R

In [ ]:
def monta_pipeline_modelo_s(regressor: RegressorMixin):
    tranformed_regressor = TransformedTargetRegressor(
        regressor=regressor, transformer=MinMaxScaler())

    pipeline_modelo_1 = Pipeline([
        ('normaliza_perametros', MinMaxScaler()),
        ('transform', tranformed_regressor)
    ]
    )
    return pipeline_modelo_1


def corte_treino_modelo_s(description: DataFrame, parametros_voo: DataFrame, dados_sensores: DataFrame, num_ciclos_iniciais: int):
    ciclos_inicias = description['cycle'] <= num_ciclos_iniciais
    corte_parametros_voo = parametros_voo[ciclos_inicias]
    corte_dados_sensores = dados_sensores[ciclos_inicias]
    return corte_parametros_voo, corte_dados_sensores

ParametrosTreino = namedtuple('ParametrosTreino',
                              field_names=['num_cortes', 'num_iteracoes', 'usar_todos_cores',
                                           'distribuicoes', 'num_cores'],
                              defaults=[5, 5, True, {}, None])


def treina_modelo_s(pipeline_modelo_s: Pipeline, parametros: ParametrosTreino,
                    corte_parametros_voo: DataFrame, corte_dados_sensores: DataFrame):
    num_cores = -1 if parametros.usar_todos_cores else parametros.num_cores
    treino_aleatorio = RandomizedSearchCV(pipeline_modelo_s, param_distributions=parametros.distribuicoes,
                                          cv=KFold(n_splits = parametros.num_cortes, shuffle=True),
                                          # cv=parametros.num_cortes,
                                          n_iter=parametros.num_iteracoes, verbose=3,
                                          random_state=42, n_jobs=num_cores, scoring='neg_root_mean_squared_error')

    treino_aleatorio.fit(corte_parametros_voo, corte_dados_sensores)

    return treino_aleatorio


def calcula_distancia_por_ciclo(dados_sensores: DataFrame,
                                dados_simulados: DataFrame, A: DataFrame, Y: DataFrame = None):
    delta_quadrado = (dados_sensores - dados_simulados).pow(2)
    grupo = ['unit', 'cycle']
    datasets = [ A[grupo], delta_quadrado ]
    if Y is not None:
      datasets.append(Y)
    X = pd.concat(datasets, axis = 1).groupby(grupo, group_keys=False)
    media = X.mean().reset_index()
    media[delta_quadrado.columns] = media[delta_quadrado.columns].pow(0.5)
    return media


def treina_modelo_r(algoritmo: RegressorMixin, parametros: ParametrosTreino,
                    delta_ciclos: DataFrame, RUL: DataFrame):
    num_cores = -1 if parametros.usar_todos_cores else parametros.num_cores
    treino_aleatorio = RandomizedSearchCV(algoritmo, param_distributions=parametros.distribuicoes,
                                          cv=KFold(n_splits = parametros.num_cortes, shuffle=True),
                                          # cv=parametros.num_cortes,
                                          n_iter=parametros.num_iteracoes, verbose=3,
                                          random_state=42, n_jobs=num_cores, scoring='neg_root_mean_squared_error')

    treino_aleatorio.fit(delta_ciclos, RUL)

    return treino_aleatorio

### 04.05 - Treino modelo sensores (1a fase)

In [ ]:
corte_parametros_voo, corte_dados_sensores = corte_treino_modelo_s(A_dev, W_dev,
                                                                   X_s_dev, num_ciclos_iniciais=5)

In [ ]:
# Quais algoritmos e modelos devemos treinar?

# Random Forest
treinar_modelo_sensor_rf = False
treinar_modelo_rul_rf = False
# Linear regression
treinar_modelo_sensor_linear = False
treinar_modelo_rul_linear = False
# Ridge linear regression
treinar_modelo_sensor_ridge = False
treinar_modelo_rul_ridge = False

if treinar_modelo_sensor_rf or treinar_modelo_sensor_linear or treinar_modelo_sensor_ridge or treinar_modelo_rul_rf or treinar_modelo_rul_linear or treinar_modelo_rul_ridge:
  assert usar_gdrive, "É necessário utilizar o gdrive para salvar os modelos"

In [ ]:
# Random Forest

# no caso da RF, é melhor usar apenas 1 instancia e não todos os cores.
# caso contrário a VM para de funcionar por memória.

# também não é possível fazer muitas iterações, pois o Colab tem limite de tempo.

if treinar_modelo_sensor_rf:
    print_data()

    pipeline_s_rf = monta_pipeline_modelo_s(RandomForestRegressor(max_depth = 10, n_jobs = -1))

    parametros_s_rf = ParametrosTreino(usar_todos_cores = False, num_iteracoes=4, distribuicoes = {
        'transform__regressor__n_estimators': list(range(500, 3001, 250)),
    })

    modelo_sensor_rf = treina_modelo_s(
        pipeline_s_rf, parametros_s_rf, corte_parametros_voo, corte_dados_sensores)
    print_data()
    salvar_modelo(f'modelo_sensor_random_forest_{len(corte_dados_sensores.columns)}features', modelo_sensor_rf)
    print_data()
 

In [ ]:
# Linear regression

if treinar_modelo_sensor_linear:
    pipeline_s_lr = monta_pipeline_modelo_s(LinearRegression())
    
    parametros_s_lr = ParametrosTreino(num_cortes=10)

    modelo_sensor_linear = treina_modelo_s(
        pipeline_s_lr, parametros_s_lr, corte_parametros_voo, corte_dados_sensores)
    salvar_modelo(f'modelo_sensor_linear_{len(corte_dados_sensores.columns)}features', modelo_sensor_linear)


In [ ]:
# Ridge linear regression

if treinar_modelo_sensor_ridge:
    pipeline_rlr = monta_pipeline_modelo_s(Ridge())

    parametros_rlr = ParametrosTreino(num_iteracoes = 10, num_cortes=10, distribuicoes = {
        'transform__regressor__alpha': loguniform(1e-4, 1e2),
    })

    modelo_sensor_ridge = treina_modelo_s(
        pipeline_rlr, parametros_rlr, corte_parametros_voo, corte_dados_sensores)
    salvar_modelo(f'modelo_sensor_ridge_{len(corte_dados_sensores.columns)}features', modelo_sensor_ridge)


In [ ]:
# teste dos modelos S

# modelos S com todos os sensores (sem feature cycle e apenas 5 primeiros voos)
binario_modelo_sensor_rf_14f  = '/20230415_095625_modelo_sensor_random_forest_14features.joblib'
binario_modelo_sensor_linear_14f  = '/20230415_134506_modelo_sensor_linear_14features.joblib'
binario_modelo_sensor_ridge_14f = '/20230415_134542_modelo_sensor_ridge_14features.joblib'

if carregar_modelos_github:
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230415_095625_modelo_sensor_random_forest_14features.joblib > $binario_modelo_sensor_rf_14f
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230415_134506_modelo_sensor_linear_14features.joblib > $binario_modelo_sensor_linear_14f
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230415_134542_modelo_sensor_ridge_14features.joblib > $binario_modelo_sensor_ridge_14f
else:
    assert usar_gdrive, "É necessário utilizar o gdrive para carregar os modelos"
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_sensor_rf_14f /
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_sensor_linear_14f /
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_sensor_ridge_14f /

modelo_sensor_rf = load(binario_modelo_sensor_rf_14f)
modelo_sensor_linear = load(binario_modelo_sensor_linear_14f)
modelo_sensor_ridge = load(binario_modelo_sensor_ridge_14f)


In [ ]:
modelo_sensor_rf.best_estimator_.named_steps.transform.regressor

In [ ]:
## para escolha do melhor algoritmo para o modelo S, verificamos qual tem o melhor RMSE para os dados de teste

corte_parametros_voo_test, corte_dados_sensores_test = corte_treino_modelo_s(A_test, W_test, X_s_test, num_ciclos_iniciais=5)

print_scores('Sensor - RF - test', corte_dados_sensores_test, modelo_sensor_rf.predict(corte_parametros_voo_test))
print_scores('Sensor - Linear - test', corte_dados_sensores_test, modelo_sensor_linear.predict(corte_parametros_voo_test))
print_scores('Sensor - Ridge - test', corte_dados_sensores_test, modelo_sensor_ridge.predict(corte_parametros_voo_test))

### 04.06 - Treino modelo RUL -  (1a fase)

In [ ]:
X_predict_dev = modelo_sensor_rf.predict(W_dev)
# define o tipo de predição (para salvar no nome)
# tipo_sensor = 'rg'
tipo_sensor = 'rf'

In [ ]:
delta_ciclo = calcula_distancia_por_ciclo(X_s_dev, X_predict_dev, A_dev, Y_dev)

delta_RUL = delta_ciclo['RUL']
delta_ciclo = delta_ciclo.drop(columns=['unit', 'cycle', 'RUL'])

In [ ]:
if treinar_modelo_rul_rf:
    pipeline_r_rf = RandomForestRegressor(max_depth = 10, n_jobs = -1)

    parametros_r_rf = ParametrosTreino(usar_todos_cores = False, distribuicoes = {
        'n_estimators': [ x for x in range(300, 4001, 300) ],
    })

    modelo_rul_rf = treina_modelo_r(pipeline_r_rf, parametros_r_rf, delta_ciclo, delta_RUL)
    salvar_modelo(f'modelo_rul_s_{tipo_sensor}_r_rf_{len(delta_ciclo.columns)}feat', modelo_rul_rf)

In [ ]:
# ps = pd.Series(modelo_rul_rf.best_estimator_.feature_importances_, index = modelo_rul_rf.best_estimator_.feature_names_in_)
# (ps * 100).sort_values(ascending=False)[0:5].sum()
# ps.sort_values(ascending=False)

In [ ]:
if treinar_modelo_rul_linear:
    pipeline_r_lr = LinearRegression()

    parametros_r_lr = ParametrosTreino()

    modelo_url_linear = treina_modelo_r(pipeline_r_lr, parametros_r_lr, delta_ciclo, delta_RUL)
    salvar_modelo(f'modelo_rul_s_{tipo_sensor}_r_linear_{len(delta_ciclo.columns)}feat', modelo_url_linear)

In [ ]:
# Ridge linear regression

if treinar_modelo_rul_ridge:
    pipeline_r_rlr = Ridge()

    parametros_r_rlr = ParametrosTreino(num_iteracoes = 10, num_cortes=10, distribuicoes = {
        'alpha': loguniform(1e-4, 1e2),
    })

    modelo_rul_ridge = treina_modelo_r(pipeline_r_rlr, parametros_r_rlr, delta_ciclo, delta_RUL)
    salvar_modelo(f'modelo_rul_s_{tipo_sensor}_r_ridge_{len(delta_ciclo.columns)}feat', modelo_rul_ridge)

In [ ]:
X_predict_test = modelo_sensor_rf.predict(W_test)


In [ ]:
delta_ciclo_test = calcula_distancia_por_ciclo(X_s_test, X_predict_test, A_test, Y_test)

delta_RUL_test = delta_ciclo_test['RUL']
unidade_test = delta_ciclo_test['unit']
delta_ciclo_test = delta_ciclo_test.drop(columns=['unit', 'cycle', 'RUL'])

In [ ]:
#######

# # usando RF para sensores
binario_modelo_rul_rf_14f  = '/20230428_214436_modelo_rul_s_rf_r_rf_14feat.joblib'
binario_modelo_rul_linear_14f  = '/20230428_215754_modelo_rul_s_rf_r_linear_14feat.joblib'
binario_modelo_rul_ridge_14f = '/20230428_215757_modelo_rul_s_rf_r_ridge_14feat.joblib'

if carregar_modelos_github:
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230428_214436_modelo_rul_s_rf_r_rf_14feat.joblib > $binario_modelo_rul_rf_14f
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230428_215754_modelo_rul_s_rf_r_linear_14feat.joblib > $binario_modelo_rul_linear_14f
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230428_215757_modelo_rul_s_rf_r_ridge_14feat.joblib > $binario_modelo_rul_ridge_14f
else:
    assert usar_gdrive, "É necessário utilizar o gdrive para carregar os modelos"
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_rul_rf_14f /
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_rul_linear_14f /
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_rul_ridge_14f /

modelo_rul_rf = load(binario_modelo_rul_rf_14f)
modelo_rul_linear = load(binario_modelo_rul_linear_14f)
modelo_rul_ridge = load(binario_modelo_rul_ridge_14f)


In [ ]:
delta_ciclo_test_pred_rf  = modelo_rul_rf.predict(delta_ciclo_test)
delta_ciclo_test_pred_lr  = modelo_rul_linear.predict(delta_ciclo_test)
delta_ciclo_test_pred_rlr = modelo_rul_ridge.predict(delta_ciclo_test)

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_rf)

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_lr)

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_rlr)

In [ ]:
print_scores('R - RF - Test', delta_RUL_test, delta_ciclo_test_pred_rf)
print_scores('R - LR - Test', delta_RUL_test, delta_ciclo_test_pred_lr)
print_scores('R - RLR - Test', delta_RUL_test, delta_ciclo_test_pred_rlr)

In [ ]:
ps = pd.Series(modelo_rul_rf.best_estimator_.feature_importances_, index = modelo_rul_rf.best_estimator_.feature_names_in_)
ps.sort_values(ascending=False)[0:5].index

### 04.07 - Treino modelo sensor (2a fase)

In [ ]:
# Quais algoritmos e modelos devemos treinar?

# Random Forest
treinar_modelo_sensor_rf = False
treinar_modelo_rul_rf = False
# Linear regression
treinar_modelo_sensor_linear = False
treinar_modelo_rul_linear = False
# Ridge linear regression
treinar_modelo_sensor_ridge = False
treinar_modelo_rul_ridge = False

In [ ]:
parametros = ['Total temperature at LPT outlet', 'Total temperature at HPT outlet',
       'Total pressure at LPT outlet', 'Fuel flow', 'Physical fan speed']

corte_parametros_voo, corte_dados_sensores = corte_treino_modelo_s(A_dev, W_dev, X_s_dev[parametros], num_ciclos_iniciais=5)

In [ ]:
# Random Forest

# no caso da RF, é melhor usar apenas 1 instancia e não todos os cores.
# caso contrário a VM para de funcionar por memória.

# também não é possível fazer muitas iterações, pois o Colab tem limite de tempo.

if treinar_modelo_sensor_rf:
    print_data()

    pipeline_s_rf = monta_pipeline_modelo_s(RandomForestRegressor(max_depth = 10, n_jobs = -1))

    parametros_s_rf = ParametrosTreino(usar_todos_cores = False, num_iteracoes=4, distribuicoes = {
        'transform__regressor__n_estimators': list(range(500, 3001, 250)),
    })

    modelo_sensor_rf = treina_modelo_s(
        pipeline_s_rf, parametros_s_rf, corte_parametros_voo, corte_dados_sensores)
    print_data()
    salvar_modelo(f'modelo_sensor_random_forest_{len(corte_dados_sensores.columns)}features', modelo_sensor_rf)
    print_data()
 

In [ ]:
# Linear regression

if treinar_modelo_sensor_linear:
    pipeline_s_lr = monta_pipeline_modelo_s(LinearRegression())
    
    parametros_s_lr = ParametrosTreino(num_cortes=10)

    modelo_sensor_linear = treina_modelo_s(
        pipeline_s_lr, parametros_s_lr, corte_parametros_voo, corte_dados_sensores)
    salvar_modelo(f'modelo_sensor_linear_{len(corte_dados_sensores.columns)}features', modelo_sensor_linear)


In [ ]:
# Ridge linear regression

if treinar_modelo_sensor_ridge:
    pipeline_rlr = monta_pipeline_modelo_s(Ridge())

    parametros_rlr = ParametrosTreino(num_iteracoes = 10, num_cortes=10, distribuicoes = {
        'transform__regressor__alpha': loguniform(1e-4, 1e2),
    })

    modelo_sensor_ridge = treina_modelo_s(
        pipeline_rlr, parametros_rlr, corte_parametros_voo, corte_dados_sensores)
    salvar_modelo(f'modelo_sensor_ridge_{len(corte_dados_sensores.columns)}features', modelo_sensor_ridge)


In [ ]:
# teste dos modelos S

# modelos S com todos os sensores (sem feature cycle e apenas 5 primeiros voos)
binario_modelo_sensor_rf_5f  = '/20230429_070900_modelo_sensor_random_forest_5features.joblib'
binario_modelo_sensor_linear_5f  = '/20230429_070913_modelo_sensor_linear_5features.joblib'
binario_modelo_sensor_ridge_5f = '/20230429_070923_modelo_sensor_ridge_5features.joblib'


if carregar_modelos_github:
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230429_070900_modelo_sensor_random_forest_5features.joblib > $binario_modelo_sensor_rf_5f
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230429_070913_modelo_sensor_linear_5features.joblib > $binario_modelo_sensor_linear_5f
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230429_070923_modelo_sensor_ridge_5features.joblib > $binario_modelo_sensor_ridge_5f
else:
    assert usar_gdrive, "É necessário utilizar o gdrive para carregar os modelos"
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_sensor_rf_5f /
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_sensor_linear_5f /
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_sensor_ridge_5f /

modelo_5sensor_rf = load(binario_modelo_sensor_rf_5f)
modelo_5sensor_linear = load(binario_modelo_sensor_linear_5f)
modelo_5sensor_ridge = load(binario_modelo_sensor_ridge_5f)


In [ ]:
## para escolha do melhor algoritmo para o modelo S, verificamos qual tem o melhor RMSE para os dados de teste

corte_parametros_voo_test, corte_dados_sensores_test = corte_treino_modelo_s(A_test, W_test, X_s_test[parametros], num_ciclos_iniciais=5)

print_scores('Sensor - RF - test', corte_dados_sensores_test, modelo_5sensor_rf.predict(corte_parametros_voo_test))
print_scores('Sensor - Linear - test', corte_dados_sensores_test, modelo_5sensor_linear.predict(corte_parametros_voo_test))
print_scores('Sensor - Ridge - test', corte_dados_sensores_test, modelo_5sensor_ridge.predict(corte_parametros_voo_test))

### 04.08 - Treino modelo RUL (2a fase)

In [ ]:
X_predict_dev = modelo_5sensor_rf.predict(W_dev)
# define o tipo de predição (para salvar no nome)
tipo_sensor = 'rf'

In [ ]:
delta_ciclo = calcula_distancia_por_ciclo(X_s_dev[parametros], X_predict_dev, A_dev, Y_dev)

delta_RUL = delta_ciclo['RUL']
delta_ciclo = delta_ciclo.drop(columns=['unit', 'cycle', 'RUL'])

In [ ]:
if treinar_modelo_rul_rf:
    pipeline_r_rf = RandomForestRegressor(max_depth = 10, n_jobs = -1)

    parametros_r_rf = ParametrosTreino(usar_todos_cores = False, distribuicoes = {
        'n_estimators': [ x for x in range(300, 4001, 300) ],
    })

    modelo_rul_rf = treina_modelo_r(pipeline_r_rf, parametros_r_rf, delta_ciclo, delta_RUL)
    salvar_modelo(f'modelo_rul_s_{tipo_sensor}_r_rf_{len(delta_ciclo.columns)}feat', modelo_rul_rf)

In [ ]:
if treinar_modelo_rul_linear:
    pipeline_r_lr = LinearRegression()

    parametros_r_lr = ParametrosTreino()

    modelo_url_linear = treina_modelo_r(pipeline_r_lr, parametros_r_lr, delta_ciclo, delta_RUL)
    salvar_modelo(f'modelo_rul_s_{tipo_sensor}_r_linear_{len(delta_ciclo.columns)}feat', modelo_url_linear)

In [ ]:
# Ridge linear regression

if treinar_modelo_rul_ridge:
    pipeline_r_rlr = Ridge()

    parametros_r_rlr = ParametrosTreino(num_iteracoes = 10, num_cortes=10, distribuicoes = {
        'alpha': loguniform(1e-4, 1e2),
    })

    modelo_rul_ridge = treina_modelo_r(pipeline_r_rlr, parametros_r_rlr, delta_ciclo, delta_RUL)
    salvar_modelo(f'modelo_rul_s_{tipo_sensor}_r_ridge_{len(delta_ciclo.columns)}feat', modelo_rul_ridge)

In [ ]:
X_predict_test = modelo_5sensor_rf.predict(W_test)


In [ ]:
modelo_5sensor_rf.best_estimator_.named_steps.transform.regressor

In [ ]:
delta_ciclo_test = calcula_distancia_por_ciclo(X_s_test[parametros], X_predict_test, A_test, Y_test)

delta_RUL_test = delta_ciclo_test['RUL']
unidade_test = delta_ciclo_test['unit']
delta_ciclo_test = delta_ciclo_test.drop(columns=['unit', 'cycle', 'RUL'])

In [ ]:
#######

# # usando RF para sensores
binario_modelo_rul_rf_5f  = '/20230429_143436_modelo_rul_s_rf_r_rf_5feat.joblib'
binario_modelo_rul_linear_5f  = '/20230429_143437_modelo_rul_s_rf_r_linear_5feat.joblib'
binario_modelo_rul_ridge_5f = '/20230429_143438_modelo_rul_s_rf_r_ridge_5feat.joblib'

if carregar_modelos_github:
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230429_143436_modelo_rul_s_rf_r_rf_5feat.joblib > $binario_modelo_rul_rf_5f
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230429_143437_modelo_rul_s_rf_r_linear_5feat.joblib > $binario_modelo_rul_linear_5f
    !curl -L https://github.com/leomarssilva/projeto-final-engenharia-uff/releases/download/v0.0.1/20230429_143438_modelo_rul_s_rf_r_ridge_5feat.joblib > $binario_modelo_rul_ridge_5f
else:
    assert usar_gdrive, "É necessário utilizar o gdrive para carregar os modelos"
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_rul_rf_5f /
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_rul_linear_5f /
    !rsync -h --progress /content/drive/MyDrive/tcc-machine-learning/$binario_modelo_rul_ridge_5f /

modelo_5rul_rf = load(binario_modelo_rul_rf_5f)
modelo_5rul_linear = load(binario_modelo_rul_linear_5f)
modelo_5rul_ridge = load(binario_modelo_rul_ridge_5f)


In [ ]:
delta_ciclo_test_pred_rf5  = modelo_5rul_rf.predict(delta_ciclo_test)
delta_ciclo_test_pred_lr5  = modelo_5rul_linear.predict(delta_ciclo_test)
delta_ciclo_test_pred_rlr5 = modelo_5rul_ridge.predict(delta_ciclo_test)

In [ ]:
print_scores('R - RF - Test', delta_RUL_test, delta_ciclo_test_pred_rf5)
print_scores('R - LR - Test', delta_RUL_test, delta_ciclo_test_pred_lr5)
print_scores('R - RLR - Test', delta_RUL_test, delta_ciclo_test_pred_rlr5)

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_rf, delta_ciclo_test_pred_rf5)

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_lr, delta_ciclo_test_pred_lr5)

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_rlr, delta_ciclo_test_pred_rlr5)

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_lr, delta_ciclo_test_pred_rlr, ["linear", "ridge"])

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_lr5, delta_ciclo_test_pred_rlr5, ["linear", "ridge"])

In [ ]:
modelo_rul_rf.best_estimator_

In [ ]:
modelo_5rul_rf.best_estimator_

In [ ]:
modelo_rul_ridge.estimator.alpha

In [ ]:
modelo_5rul_ridge.estimator.alpha

In [ ]:
plot_motores(delta_RUL_test, delta_ciclo_test_pred_rf5, delta_ciclo_test_pred_lr5, ["F. aleatória", "R. linear"])